In [1]:
### load modules

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'


from qa_utils import *

# reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Merian data repo
root = '/projects/MERIAN/repo'
butler = dafButler.Butler(root)

In [3]:
import lsst.daf.butler as dafButler
import scipy.stats as stats
from astropy.io import ascii
from astropy.table import Table
import numpy as np

 


def mad_sigmaclip(array, low=3., high=3.):
    '''
    similar to sigmaclip, but using median and MAD instead of mean and std
    ''' 
    # mask=0 means the value should be discarded
    
    # clip nan
    mask = np.ones(array.shape,dtype=int)
    mask[np.isnan(array)] = 0
    c = array[mask==1]
    
    delta = 1
    while delta:
        c_std = stats.median_abs_deviation(array)
        c_median =  np.nanmedian(c)
        size = c.size
        critlower = c_median - c_std * low
        critupper = c_median + c_std * high
        mask[(array <= critlower) | (array >= critupper)] = 0
        c = c[(c >= critlower) & (c <= critupper)]
        delta = size - c.size

    return mask, array[mask==1]


# random tract
list_tract_ids = [8280,8525,8764,8769,9010,9078,9088,9099,9102,9131,9135,9227,9313,9319,9344,9378,9456,9459,9470,9556,9564,9574,9589,9798,9800,9812,9828,9939,9946,9953,10040,10048,10061,10183,10293,10427]  

collection = 'DECam/runs/merian/dr1_wide'
skymap = 'hsc_rings_v1'

flux_colname = 'N540_gaap1p5Flux'
tract_id_list = []
tract_median_list = []
tract_std_list = []

for i,tract in enumerate(list_tract_ids):
    butler = dafButler.Butler('/projects/MERIAN/repo')
    dataId = dict(tract=tract)


    try:
        
        cat_coadd = butler.get( 'objectTable_tract', tract=tract, instrument='DECam',
                        skymap=skymap, collections=collection)
        
        print(f'Has ObjectTable_tract for {tract}')
    
    except:
        print(f'No objectTable_tract for {tract}')
        continue

    ### sky objects
    skyobj_cat = cat_coadd[cat_coadd.merge_peak_sky==True]
    
    ### Tract Level
    flux = cat_coadd[flux_colname]
    mask, flux_clipped = mad_sigmaclip(flux,low=3,high=3)
    Neff = np.sum(mask)
    
    
    median = np.nanmedian(flux_clipped)
    std = stats.median_abs_deviation(flux_clipped)

    tract_id_list.append(tract)
    tract_median_list.append(median)
    tract_std_list.append(std)
    

    ### Patch Level
    if i==0:
        patch_list = np.unique(cat_coadd['patch'])
        patch_median_list = []
        patch_std_list = []

        for patch_id in patch_list:
            patch_cat = cat_coadd[cat_coadd['patch']==patch_id]
            patch_flux_raw = patch_cat[flux_colname]
            mask, patch_flux = sigmaclip(patch_flux_raw)
            Neff = len(patch_flux)
            
            mid_flux = np.nanmedian(patch_flux)
            std_flux = np.sqrt(np.nanmean((patch_flux-mid_flux)**2))
            
            patch_median_list.append(mid_flux)
            patch_std_list.append(std_flux)

        
        patch_stat = Table([patch_list,patch_median_list,patch_std_list],names=['patch','median','std'])
        patch_stat['patch'].description = 'patch id'
        patch_stat['median'].description = f'median of {flux_colname} flux for each patch'
        patch_stat['std'].description = f'std of {flux_colname} flux for each patch'
        patch_stat.meta['tract'] = tract

        patch_stat.meta['comments'] = f'statisitcs of {flux_colname} flux for each patch in tract {tract}'

        ascii.write(patch_stat, f'patch_stat_{tract}.ecsv', overwrite=True, format='ecsv')
    
    del cat_coadd
    del skyobj_cat
    del flux
    del mask
    del flux_clipped




tract_stat = Table([tract_id_list,tract_median_list,tract_std_list],names=['tract','median','std'])
tract_stat['tract'].description = 'tract id'
tract_stat['median'].description = f'median of {flux_colname} flux for each tract'
tract_stat['std'].description = f'std of {flux_colname} flux for each tract'

tract_stat.meta['comments'] = f'statisitcs of {flux_colname} flux for each tract'

ascii.write(tract_stat, f'tract_stat.ecsv', overwrite=True, format='ecsv')



Has ObjectTable_tract for 8280


/scratch/gpfs/HSC/LSST/stack_20230302/conda/envs/lsst-scipipe-4.0.1/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Has ObjectTable_tract for 8525


/scratch/gpfs/HSC/LSST/stack_20230302/conda/envs/lsst-scipipe-4.0.1/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Has ObjectTable_tract for 8764


/scratch/gpfs/HSC/LSST/stack_20230302/conda/envs/lsst-scipipe-4.0.1/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No objectTable_tract for 8769
